# 洗錢
- 筆記: https://www.notion.so/jayschsu/c6a6219dd004469bbbfbecfb6d9883f3

In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
import os
import pandas as pd
import numpy as np 
from itertools import product
from joblib import Parallel, delayed
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import time
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, RandomOverSampler, ADASYN, KMeansSMOTE, SMOTEN, SMOTENC, SVMSMOTE
from imblearn.under_sampling import TomekLinks
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score, recall_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from collections import namedtuple
import joblib

SCORE_TUPLE = namedtuple('SCORE_TUPLE', 'feature, preprocess, sampling, model, iter, precision, time')

pd.set_option('display.max_rows', 500)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)


In [2]:
from data import Data
from log import init_logging

In [3]:
init_logging('data')

In [4]:
dataset = Data()

2022-10-31 09:14:03,689 INFO: func:init_data: 2.66 sec (utils.py:16)
2022-10-31 09:14:03,690 INFO: custinfo_preprocess (data.py:52)
invalid_data_qty: 2
2022-10-31 09:14:03,799 INFO: func:custinfo_preprocess: 0.11 sec (utils.py:16)
2022-10-31 09:14:03,800 INFO: featuring_alertDate (data.py:81)
2022-10-31 09:14:29,345 INFO: func:featuring_alertDate: 25.55 sec (utils.py:16)
2022-10-31 09:14:29,346 INFO: featuring_accumulate (data.py:147)


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-2)]: Batch computation too fast (0.1918s.) Setting batch_size=2.
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-2)]: Done  37 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-2)]: Batch computation too fast (0.0439s.) Setting batch_size=4.
[Parallel(n_jobs=-2)]: Done  59 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-2)]: Batch computation too fast (0.0872s.) Setting batch_size=8.
[Parallel(n_jobs=-2)]: Done  99 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-2)]: Batch computation too fast (0.1565s.) Setting batch_size=16.
[Parallel(n_jobs=-2)]: Done 183 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-2)]: Done 343 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-2)]: Done 583 t

2022-10-31 09:15:08,912 INFO: func:featuring_accumulate: 39.57 sec (utils.py:16)
2022-10-31 09:15:08,914 INFO: featuring_integrate (data.py:162)
2022-10-31 09:15:30,572 INFO: func:featuring_integrate: 21.66 sec (utils.py:16)


### Data
- dp_df
    - DEBIT是進錢(+) & CREDIT是出錢(-) 

In [ ]:
ccba_df = pd.read_csv('data/dataset1/public_train_x_ccba_full_hashed.csv')
cdtx_df = pd.read_csv('data/dataset1/public_train_x_cdtx0001_full_hashed.csv')
custinfo_df = pd.read_csv('data/dataset1/public_train_x_custinfo_full_hashed.csv')
dp_df = pd.read_csv('data/dataset1/public_train_x_dp_full_hashed.csv')
remit_df = pd.read_csv('data/dataset1/public_train_x_remit1_full_hashed.csv')
tr_alertX_df = pd.read_csv('data/dataset1/train_x_alert_date.csv')
tr_sarY_df = pd.read_csv('data/dataset1/train_y_answer.csv')
public_alertX_df = pd.read_csv('data/dataset1/public_x_alert_date.csv')

**Preprocess**
- 將alert_date, sar_flag merge到custinfo

In [ ]:
#mapping alert_date & sar flag
alertDate_dict = tr_alertX_df.set_index('alert_key').to_dict()['date']
sarFlag_dict = tr_sarY_df.set_index('alert_key').to_dict()['sar_flag']
public_alertX_df['sar_flag']=2
alertDatePublc_dict = public_alertX_df.set_index('alert_key').to_dict()['date']
sarFlagPublic_dict = public_alertX_df.set_index('alert_key').to_dict()['sar_flag']
custinfo_df['alert_date'] = custinfo_df['alert_key'].map(lambda x: alertDate_dict[x] if x in alertDate_dict else alertDatePublc_dict[x] )
custinfo_df['sar_flag'] = custinfo_df['alert_key'].map(lambda x: sarFlag_dict[x] if x in sarFlag_dict else sarFlagPublic_dict[x])
custinfo_df['sar_flag_nunique'] = custinfo_df.groupby(['cust_id','alert_date'])['sar_flag'].transform('nunique')
invalid_data_qty = sum(custinfo_df['sar_flag_nunique']>1)
print(f'invalid_data_qty: {invalid_data_qty}')
custinfo_valid = custinfo_df[custinfo_df['sar_flag_nunique']==1]
custinfo_valid.drop_duplicates(['cust_id','alert_date','sar_flag'], inplace=True)
custinfo_valid = custinfo_valid.set_index(['cust_id','alert_date'])


## Feturing

**1. by Date**


In [ ]:
#cdtx
cdtxDate_ = cdtx_df.groupby(['cust_id','date']).agg(cdtxAmt_ADate=('amt', sum), cdtxCnt_ADate=('date', 'count'))
cdtxDate_ntd = cdtx_df[cdtx_df['cur_type']==47].groupby(['cust_id','date']).agg(cdtxAmtNTD_ADate=('amt', sum), cdtxCntNTD_ADate=('date', 'count'))
cdtxDate_fc = cdtx_df[cdtx_df['cur_type']!=47].groupby(['cust_id','date']).agg(cdtxAmtFC_ADate=('amt', sum), cdtxCntFC_ADate=('date', 'count'))
cdtxDate_tw = cdtx_df[cdtx_df['country']==130].groupby(['cust_id','date']).agg(cdtxAmtTW_ADate=('amt', sum), cdtxCntTW_ADate=('date', 'count'))
cdtxDate_f = cdtx_df[cdtx_df['country']!=130].groupby(['cust_id','date']).agg(cdtxAmtF_ADate=('amt', sum), cdtxCntF_ADate=('date', 'count'))
cdtxDate_df = pd.concat([cdtxDate_, cdtxDate_ntd, cdtxDate_fc, cdtxDate_tw, cdtxDate_f], axis=1)

#dp
dp_df['amt'] = dp_df['tx_amt']*dp_df['exchg_rate']
dpDate_ = dp_df.groupby(['cust_id','tx_date']).agg(dpAmt_ADate=('tx_amt', sum), dpCnt_ADate=('tx_date', 'count'))
dpDate_CR = dp_df[dp_df['debit_credit']=='CR'].groupby(['cust_id','tx_date']).agg(dpAmtCR_ADate=('tx_amt', sum), dpCntCR_ADate=('tx_date', 'count'))
dpDate_DB = dp_df[dp_df['debit_credit']=='CB'].groupby(['cust_id','tx_date']).agg(dpAmtDB_ADate=('tx_amt', sum), dpCntDB_ADate=('tx_date', 'count'))
dpDate_CC = dp_df[(dp_df['tx_type']==1)&(dp_df['info_asset_code']==12)].groupby(['cust_id','tx_date']).agg(dpAmtCC_ADate=('tx_amt', sum), dpCntCC_ADate=('tx_date', 'count'))
dpDate_NCC = dp_df[~((dp_df['tx_type']==1)&(dp_df['info_asset_code']==12))].groupby(['cust_id','tx_date']).agg(dpAmtNCC_ADate=('tx_amt', sum), dpCntNCC_ADate=('tx_date', 'count'))
dpDate_CBank = dp_df[dp_df['cross_bank']==1].groupby(['cust_id','tx_date']).agg(dpAmtCBank_ADate=('tx_amt', sum), dpCntCBank_ADate=('tx_date', 'count'))
dpDate_InBank = dp_df[dp_df['cross_bank']==0].groupby(['cust_id','tx_date']).agg(dpAmtInBank_ADate=('tx_amt', sum), dpCntInBank_ADate=('tx_date', 'count'))
dpDate_ATM = dp_df[dp_df['ATM']==1].groupby(['cust_id','tx_date']).agg(dpAmtATM_ADate=('tx_amt', sum), dpCntATM_ADate=('tx_date', 'count'))
dpDate_NATM = dp_df[dp_df['ATM']==0].groupby(['cust_id','tx_date']).agg(dpAmtNATM_ADate=('tx_amt', sum), dpCntNATM_ADate=('tx_date', 'count'))
dpDate_branchNunique = dp_df.groupby(['cust_id','tx_date']).agg(dpBranchNunique_ADate=('txbranch', 'nunique'))
dpDate_df = pd.concat([dpDate_, dpDate_CR, dpDate_DB, dpDate_CC, dpDate_NCC, dpDate_CBank, dpDate_InBank, dpDate_ATM, dpDate_NATM, dpDate_branchNunique], axis=1)

#remit
remitDate_df = remit_df.groupby(['cust_id','trans_date']).agg(remitAmt_ADate=('trade_amount_usd', sum), remitCnt_ADate=('trans_no', 'count'))



**2. Accumulate recent K days**   
- accumulate feature: 整合前K天的feature
- Process: 每一個customer要分別做以下的feature engineering
    - 1. 組合cust_id * dates
    - 2. merge with date feature
    - 3. calculate accum feature
    - 4. merge date and accum features


In [ ]:

def get_accum_feature(cust_data, cust_id, date_colName, rolling_window):
    dates = list(range(cust_data.index.min(), cust_data.index.max()+1))
    all_custDate = pd.DataFrame(list(product([cust_id], dates)), columns=['cust_id', date_colName]).set_index(['cust_id', date_colName])
    cust_fullDate = pd.merge(cust_data, all_custDate, how='outer', left_index=True, right_index=True)
    cust_fullDate.sort_index(ascending=True, inplace=True)
    cust_fullDate.fillna(0, inplace=True)
    
    cust_recentAccum = cust_fullDate.rolling(window=rolling_window, min_periods=1).mean().shift()
    cust_recentAccum2W = cust_fullDate.rolling(window=rolling_window*2, min_periods=1).mean().shift()
    cust_recentAccumDiff = cust_recentAccum*2 - cust_recentAccum2W
    cust_fullDateDiff = cust_fullDate - cust_recentAccum
    
    new_cols = []
    for c in cust_recentAccum.columns:
        new_cols.append(c.replace('_ADate', '_Accum'))
    cust_recentAccum.columns=new_cols
    
    new_cols = []
    for c in cust_recentAccumDiff.columns:
        new_cols.append(c.replace('_ADate', '_AccumDiff'))
    cust_recentAccumDiff.columns=new_cols
    
    new_cols = []
    for c in cust_fullDateDiff.columns:
        new_cols.append(c.replace('_ADate', '_ADateDiff'))
    cust_fullDateDiff.columns=new_cols        
    
    feature = pd.merge(cust_data, cust_fullDateDiff, how='left', left_index=True, right_index=True)
    feature = pd.merge(feature, cust_recentAccum, how='left', left_index=True, right_index=True)
    feature = pd.merge(feature, cust_recentAccumDiff, how='left', left_index=True, right_index=True)
    
    return feature

def accum_featuring_parallel(dateFeature, date_colName='date', rolling_window=5):
    idx_custid, _ = zip(*dateFeature.index)
    idx_custid = np.unique(idx_custid)
    accu_feature_list = Parallel(n_jobs=4, verbose=10)(delayed(get_accum_feature)(dateFeature.loc[(cust_id)], cust_id, date_colName, rolling_window) for cust_id in idx_custid) 
    accu_feature = pd.concat(accu_feature_list)
    return accu_feature

def get_recallN_Precision(y_predProb, y_true):
    y_pred_df = pd.DataFrame(list(zip(y_predProb, y_true)), columns=['predProb','trueLabel'])
    y_pred_df.sort_values(by='predProb', ascending=False, inplace=True)
    y_pred_df['idx']=list(range(len(y_pred_df)))
    idx = y_pred_df[y_pred_df['trueLabel']==1]['idx'].iloc[-2]
    precision_score = (sum(y_true)-1)/idx
    return precision_score


In [ ]:
%%time
cdtx_feature = accum_featuring_parallel(cdtxDate_df, 'date', rolling_window=7)
dp_feature = accum_featuring_parallel(dpDate_df, 'tx_date', rolling_window=7)
remit_feature = accum_featuring_parallel(remitDate_df, 'trans_date', rolling_window=7)

**3. cust profile**

In [ ]:
#ccba
ccbaProfile1 = ccba_df.replace(0, np.nan).groupby('cust_id').agg(
                               ccbalupayAmt_Year=('lupay', np.mean),\
                               ccbausgamAmt_Year=('usgam', np.mean),\
                               ccbacycamAax_Year=('cycam', np.max),\
                               ccbaclamtAmt_Year=('clamt', np.mean),\
                               ccbacsamtAmt_Year=('csamt', np.mean),\
                               ccbainamtAmt_Year=('inamt', np.mean),\
                               ccbacucsmAmt_Year=('cucsm', np.mean),\
                               ccbacucahAmt_Year=('cucah', np.mean),\
                              )
ccbaProfile2 = ccba_df.groupby('cust_id').agg(ccbalupayCnt_Year=('lupay', np.count_nonzero),\
                               ccbausgamCnt_Year=('usgam', np.count_nonzero),\
                               ccbaclamtCnt_Year=('clamt', np.count_nonzero),\
                               ccbacsamtCnt_Year=('csamt', np.count_nonzero),\
                               ccbainamtCnt_Year=('inamt', np.count_nonzero),\
                               ccbacucsmCnt_Year=('cucsm', np.count_nonzero),\
                               ccbacucahCnt_Year=('cucah', np.count_nonzero),\
                               ccbabyymmCnt_Year=('byymm', np.count_nonzero),\
                              )
ccbaProfile_df = pd.concat([ccbaProfile1, ccbaProfile2], axis=1)


new_cols = []
for c in custinfo_valid.columns:
    if c in ['alert_key', 'sar_flag', 'sar_flag_nunique']:
        new_cols.append(c)
    else:
        new_cols.append(c+'_Profile')
custinfo_valid.columns=new_cols

custProfile_df = pd.merge(custinfo_valid.reset_index(level=1), ccbaProfile_df, left_index=True, right_index=True, how='left')
custProfile_df.set_index('alert_date', append=True, inplace=True)



**3. Integrate dataset**
- moneyLundry dataset (sar_flag==1, in alert_date)
- falseAlarm dataset (sar_flag==0, in alert_date)
- public dataset (sar_flag==2, in alert_date)

In [ ]:
data_df = pd.concat([custProfile_df, cdtx_feature, dp_feature, remit_feature], axis=1)
data_df['sar_flag'] =  data_df.apply(lambda r: r['sar_flag'] if r['alert_key']==r['alert_key'] else 3, axis=1)
data_df = data_df[data_df['alert_key'].notna()]

In [ ]:
labelY = 'sar_flag'
feature_all = [x for x in data_df.columns if x not in ['alert_key', 'sar_flag', 'sar_flag_nunique']]
feature_Profile = [x for x in data_df.columns if x.split('_')[-1]=='Profile']
feature_ADate = [x for x in data_df.columns if x.split('_')[-1]=='ADate']
feature_ADateDiff = [x for x in data_df.columns if x.split('_')[-1]=='ADateDiff']
feature_Accum = [x for x in data_df.columns if x.split('_')[-1]=='Accum']
feature_AccumDiff = [x for x in data_df.columns if x.split('_')[-1]=='AccumDiff']
feature_Year = [x for x in data_df.columns if x.split('_')[-1]=='Year']
assert len(feature_all)==len(feature_Profile+feature_ADate+feature_ADateDiff+feature_Accum+feature_AccumDiff+feature_Year)



## MODEL

In [ ]:

display(data_df[labelY].value_counts().to_frame())
dataset_Tr = data_df[data_df['sar_flag']!=2]
dataset_TsPub = data_df[data_df['sar_flag']==2]
dataset_Tr.fillna(0, inplace=True)
dataset_TsPub.fillna(0, inplace=True)
print('dataset_Tr:', dataset_Tr.shape)
print('dataset_TsPub:', dataset_TsPub.shape)


In [ ]:
def train_add_hat(x, features):
    import numpy as np
    import pandas as pd
    df = x.copy() 
    q95_dict = {} 
    q5_dict = {} 
    for col in features:
        q95 = np.percentile(df[col], 90)
        q5 = np.percentile(df[col], 5) 
        q95_dict[col] = q95 
        q5_dict[col] = q5 
        b = np.array(df[col])
        c = list(map(lambda x: q95 if x > q95 else x, b))
        c = list(map(lambda x: q5 if x < q5 else x, c))        
        df = df.drop(col, axis=1)
        df[col] = c 
    return df, q95_dict, q5_dict

# 使用同一标准处理测试集
def add_hat(x, features, q95_dict, q5_dict):
    import numpy as np
    import pandas as pd
    df = x.copy()
    len_d = len(df.index)  # 测试集大小
    for col in features:
        q95 = q95_dict[col]
        q5 = q5_dict[col]
        b = np.array(df[col])
        c = list(map(lambda x:q95 if x > q95 else x, b))
        c = list(map(lambda x: q5 if x < q5 else x, c))        
        df = df.drop(col, axis=1)
        df[col] = c
    return df

## Model Training

In [ ]:
X_tr, X_val, y_tr, y_val= train_test_split(dataset_Tr[featurePDAY_Diff], dataset_Tr[labelY].values, test_size=0.2, stratify=dataset_Tr[labelY])
X_tr, q95_dict, q5_dict = train_add_hat(X_tr, featurePDAY_Diff)
X_val = add_hat(X_val, featurePDAY_Diff, q95_dict, q5_dict)


In [ ]:

pipePre = Pipeline([('sc', MinMaxScaler()), ('pca', PCA(n_components=0.999999999))])
pipeSp = imbPipeline([('sp', RandomOverSampler(sampling_strategy=0.8))])

x_tr_ = pipePre.fit_transform(X_tr)
x_val_ = pipePre.transform(X_val.copy())
x_re, y_re = pipeSp.fit_resample(x_tr_, y_tr)
#x_re, y_re = x_tr_, y_tr
model = LogisticRegression()
model = model.fit(x_re, y_re)        
#tr_score
y_pred = model.predict(x_tr_)
y_predProb = model.predict_proba(x_tr_)
y_prob = np.max(y_predProb, axis=1)
y_sarProb = [x[1] for x in y_predProb]
tr_precision = get_recallN_Precision(y_sarProb, y_tr)
tr_acc = accuracy_score(y_tr, y_pred)
tr_recall = recall_score(y_tr, y_pred)

#val_score
y_predval = model.predict(x_val_)
y_predProb = model.predict_proba(x_val_)
y_predProb = [x[1] for x in y_predProb]
precisionScore = get_recallN_Precision(y_predProb, y_val)
score_tuple = SCORE_TUPLE('PDA', 'minmax', 'smote', 'LR', '', precisionScore,0)
print(f'tr_acc: {tr_acc:.3f}, tr_precision:{tr_precision:.3f}, tr_recall:{tr_recall:.3f}')
print(score_tuple)


In [ ]:
X_tr, X_val, y_tr, y_val= train_test_split(dataset_Tr[featurePDA], dataset_Tr[labelY].values, test_size=0.2, stratify=dataset_Tr[labelY])

X_tr = X_tr[featurePDA]
X_val = X_val[featurePDA]
pipePre = Pipeline([('sc', MinMaxScaler()), ('pca', PCA(n_components=0.999999999))])
pipeSp = imbPipeline([('sp', SMOTE())])

x_tr_ = pipePre.fit_transform(X_tr)
x_val_ = pipePre.transform(X_val.copy())
x_re, y_re = pipeSp.fit_resample(x_tr_, y_tr)
model = LogisticRegression(max_iter=30)
#model = SGDClassifier(loss='log', max_iter=10)
#model = XGBClassifier(n_estimators=10, use_label_encoder=False, tree_method='gpu_hist', gpu_id=0, eval_metric='logloss')
model = model.fit(x_re, y_re)        
#tr_score
y_pred = model.predict(x_tr_)
y_predProb = model.predict_proba(x_tr_)
y_prob = np.max(y_predProb, axis=1)
y_sarProb = [x[1] for x in y_predProb]
tr_precision = get_recallN_Precision(y_sarProb, y_tr)
tr_acc = accuracy_score(y_tr, y_pred)
tr_recall = recall_score(y_tr, y_pred)
print(f'tr_acc: {tr_acc:.3f}, tr_precision:{tr_precision:.3f}, tr_recall:{tr_recall:.3f}')

#val_score
y_predProb = model.predict_proba(x_val_)
y_predProb = [x[1] for x in y_predProb]
precisionScore = get_recallN_Precision(y_predProb, y_val)
score_tuple = SCORE_TUPLE(featureName, pipePreName, pipeSpName, modelName, i, precisionScore,0)
score_list.append(score_tuple)
print(score_tuple)



#過濾中間資料
if True:
    X_tr['y_sarProb']=y_sarProb
    X_tr['y_tr']=y_tr
    threshold = X_tr.sort_values(by='y_sarProb').iloc[int(len(X_tr)*0.95)]['y_sarProb']    
    X_tr_new = X_tr[(X_tr['y_sarProb']<threshold) | (X_tr['y_tr']==1)]
    y_tr_new = X_tr_new['y_tr'].values
    del X_tr_new['y_sarProb']
    del X_tr_new['y_tr']
    del X_tr['y_sarProb']
    del X_tr['y_tr']
else:
    X_tr_new = X_tr.copy()
    
x_tr_new = pipePre.fit_transform(X_tr_new)
x_val_ = pipePre.transform(X_val.copy())
x_re, y_re = pipeSp.fit_resample(x_tr_new, y_tr_new)
model = LogisticRegression()
#model = SGDClassifier(loss='log')
#model = XGBClassifier(use_label_encoder=False, tree_method='gpu_hist', gpu_id=0, eval_metric='logloss')
model = model.fit(x_re, y_re)        

y_pred = model.predict(x_tr_new)
y_predProb = model.predict_proba(x_tr_new)
y_prob = np.max(y_predProb, axis=1)
y_sarProb = [x[1] for x in y_predProb]
tr_precision = get_recallN_Precision(y_sarProb, y_tr_new)
tr_acc = accuracy_score(y_tr_new, y_pred)
tr_recall = recall_score(y_tr_new, y_pred)


#val_score
y_predProb = model.predict_proba(x_val_)
y_predProb = [x[1] for x in y_predProb]
precisionScore = get_recallN_Precision(y_predProb, y_val)
score_tuple = SCORE_TUPLE(featureName, pipePreName, pipeSpName, modelName, i, precisionScore,0)
score_list.append(score_tuple)
print(score_tuple, f'-- shape:{X_tr.shape}->{X_tr_new.shape}->{x_tr_new.shape}->{x_re.shape}')
print(f'tr_acc: {tr_acc:.3f}, tr_precision:{tr_precision:.3f}, tr_recall:{tr_recall:.3f}')


## Hyperparameter Tuning

In [ ]:
pipePreMinMax = Pipeline([('sc', MinMaxScaler()), ('pca', PCA(n_components=0.99999))])
pipePreStd = Pipeline([('sc', StandardScaler()), ('pca', PCA(n_components=0.9))])
pipePreMMStd = Pipeline([('sc1', MinMaxScaler()), ('sc2', StandardScaler()), ('pca', PCA(n_components=0.9))])
#pipe_list = zip([pipePreMinMax, pipePreStd, pipePreMMStd], ['MinMaxScaler','StandardScaler','MinMaxScaler+StandardScaler'])
pipe_list = zip([pipePreMinMax], ['MinMaxScaler'])
pipe_list = list(pipe_list)

pipeSpAdasyn = imbPipeline([('sp', ADASYN())])
pipeSpSmoten = imbPipeline([('sp', SMOTEN())])
pipeSpSvmsmote = imbPipeline([('sp', SVMSMOTE())])
pipeSpRandom = imbPipeline([('sp', RandomOverSampler())])
pipeSpSmote = imbPipeline([('sp', SMOTE())])
pipeSpBSmote = imbPipeline([('sp', BorderlineSMOTE())])
pipeSpTom = imbPipeline([('sp', TomekLinks())])
pipeSpSmoteTom = imbPipeline([('sp1', BorderlineSMOTE()), ('sp2', TomekLinks())])
sp_list = zip([pipeSpAdasyn, pipeSpSmoten, pipeSpSvmsmote, pipeSpRandom, pipeSpSmote, pipeSpBSmote, pipeSpTom, pipeSpSmoteTom], ['adaysn', 'Smoten', 'Svmsmote', 'random', 'smote','bsmote', 'tom', 'SmoteTom'])
#sp_list = zip([pipeSpSmote],['smote'])
sp_list = list(sp_list)


knn = KNeighborsClassifier(3)
svc = SVC(kernel="rbf", C=0.025, probability=True)
nusvc = NuSVC(probability=True)
dtree = DecisionTreeClassifier()
rfc = RandomForestClassifier()
adb = AdaBoostClassifier()
gdc = GradientBoostingClassifier()
gnb = GaussianNB()
lda = LinearDiscriminantAnalysis()
qda = QuadraticDiscriminantAnalysis()
sgdc = SGDClassifier(loss='log', max_iter=10)
modelLR = LogisticRegression()
modelXGBC = XGBClassifier(use_label_encoder=False, tree_method='gpu_hist', gpu_id=0, eval_metric='logloss')
modelMLPC = MLPClassifier()
#model_list = zip([qda, lda, gnb, gdc, adb, rfc, dtree, knn, modelLR, modelXGBC, modelMLPC], ['qda', 'lda', 'gnb', 'gdc', 'adb', 'rfc', 'dtree', 'knn', 'LogisticRegression', 'XGBClassifier', 'MLPClassifier'])
model_list = zip([modelLR, sgdc], ['LR', 'SGDC'])
model_list = list(model_list)

featureD = feature_ADate
featurePD = feature_Profile+feature_ADate
featurePDA = feature_Profile+feature_ADate+feature_Accum
featurePDAY = feature_Profile+feature_ADate+feature_Accum+feature_Year
featureD_Diff = feature_ADateDiff
featurePD_Diff = feature_Profile+feature_ADateDiff
featurePDA_Diff = feature_Profile+feature_ADateDiff+feature_AccumDiff
featurePDAY_Diff = feature_Profile+feature_ADateDiff+feature_AccumDiff+feature_Year
featureD_Composite = feature_ADate+feature_ADateDiff
featurePD_Composite = feature_Profile+feature_ADate+feature_ADateDiff
featurePDA_Composite = feature_Profile+feature_ADate+feature_Accum+feature_ADateDiff+feature_AccumDiff
featurePDAY_Composite = feature_Profile+feature_ADate+feature_Accum+feature_ADateDiff+feature_AccumDiff+feature_Year
feature_list = [featureD, featurePD, featurePDA, featurePDAY, featureD_Diff, featurePD_Diff, featurePDA_Diff, featurePDAY_Diff, featureD_Composite, featurePD_Composite, featurePDA_Composite, featurePDAY_Composite]
feature_list = zip(feature_list,['D', 'PD', 'PDA', 'PDAY', 'D_Diff', 'PD_Diff','PDA_Diff','PDAY_Diff','D_Composite','PD_Composite', 'PDA_Composite', 'PDAY_Composite'])
#feature_list = zip([featurePDA], ['PDA'])
feature_list = list(feature_list)

score_list_tmp=[]
for featureX, featureName in feature_list:
    for i in range(4):
        for pipePre, pipePreName in pipe_list:
            for pipeSp, pipeSpName in sp_list:
                for model, modelName in model_list:
                    score_list_tmp.append(SCORE_TUPLE(featureName, pipePreName, pipeSpName, modelName, i, 0,0))
print('hyperparameter combinations:',len(score_list_tmp))


In [ ]:
score_list = []
Y = dataset_Tr[labelY]
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=43)
skf.get_n_splits(dataset_Tr,Y)
for i, (tr_idx, val_idx) in enumerate(skf.split(dataset_Tr, Y)):
    predProb_list4 = []
    for featureX, featureName in feature_list:
        X_tr, X_val = dataset_Tr[featureX].iloc[tr_idx], dataset_Tr[featureX].iloc[val_idx]
        y_tr, y_val = Y.iloc[tr_idx], Y.iloc[val_idx]
        predProb_list3 = []
        for pipePre, pipePreName in pipe_list:
            predProb_list2 = []
            for pipeSp, pipeSpName in sp_list:
                predProb_list1 = []
                for model, modelName in model_list:
                    startTime = time.time() 
                    #preprocess
                    x_tr_ = X_tr.copy()
                    x_val_ = X_val.copy()
                    x_tr_ = pipePre.fit_transform(x_tr_)
                    x_val_ = pipePre.transform(x_val_)
                    x_re, y_re = pipeSp.fit_resample(x_tr_, y_tr)
                    model = model.fit(x_re, y_re)        
                    
                    #tr_score
                    y_pred = model.predict(x_tr_)
                    y_predProb = model.predict_proba(x_tr_)
                    y_predProb = [x[1] for x in y_predProb]
                    tr_precision = get_recallN_Precision(y_predProb, y_tr)
                    tr_acc = accuracy_score(y_tr, y_pred)
                    tr_recall = recall_score(y_tr, y_pred)
                    
                    #val score
                    y_predProb = model.predict_proba(x_val_)
                    y_predProb = [x[1] for x in y_predProb]
                    precisionScore = get_recallN_Precision(y_predProb, y_val)
                    t=time.time()-startTime
                    score_tuple = SCORE_TUPLE(featureName, pipePreName, pipeSpName, modelName, i, precisionScore, t)
                    score_list.append(score_tuple)
                    predProb_list1.append(y_predProb)
                    predProb_list2.append(y_predProb)
                    predProb_list3.append(y_predProb)
                    predProb_list4.append(y_predProb)
                    print(score_tuple)
                    print(f'-- shape:{X_tr.shape}->{x_tr_.shape}, time:{t:.1f}, ts score:{precisionScore:.4f}')
                    print(f'-- tr_acc: {tr_acc:.2f}, tr_precision:{tr_precision:.4f}, tr_recall:{tr_recall:.3f}')

                precisionScore = get_recallN_Precision(np.mean(predProb_list1, axis=0), y_val)
                score_list.append(SCORE_TUPLE(featureName, pipePreName, pipeSpName, 'ensemble', i, precisionScore, t))
            precisionScore = get_recallN_Precision(np.mean(predProb_list2, axis=0), y_val)
            score_list.append(SCORE_TUPLE(featureName, pipePreName, 'ensemble', 'ensemble', i, precisionScore, t))
        precisionScore = get_recallN_Precision(np.mean(predProb_list3, axis=0), y_val)
        score_list.append(SCORE_TUPLE(featureName, 'ensemble', 'ensemble', 'ensemble', i, precisionScore, t))
    precisionScore = get_recallN_Precision(np.mean(predProb_list4, axis=0), y_val)
    score_list.append(SCORE_TUPLE('ensemble', 'ensemble', 'ensemble', 'ensemble', i, precisionScore, t))        
                
                
score_df = pd.DataFrame(score_list)
score_df.sort_values('precision', ascending=False, inplace=True)
#joblib.dump(score_df, 'model_log/score_df.pkl')                



In [ ]:
score_df

In [ ]:

#display(score_df.sort_values('precision', ascending=False))
display(score_df.groupby(['feature'])['precision'].mean().to_frame().sort_values('precision', ascending=False))
#display(score_df.groupby(['model','iter'])['precision'].max().to_frame())
#display(score_df.groupby(['iter','model'])['precision'].max().to_frame())

In [ ]:
display(score_df.groupby('feature')['precision'].mean().to_frame().sort_values('precision', ascending=False))
display(score_df.groupby('sampling')['precision'].mean().to_frame().sort_values('precision', ascending=False))
display(score_df.groupby('model')['precision'].mean().to_frame().sort_values('precision', ascending=False))
display(score_df.groupby('iter')['precision'].mean().to_frame().sort_values('precision', ascending=False))

In [ ]:
display(score_df[score_df['model']=='LogisticRegression'].groupby('sampling')['precision'].mean().to_frame().sort_values('precision', ascending=False))
display(score_df[score_df['model']=='LogisticRegression'].groupby('feature')['precision'].mean().to_frame().sort_values('precision', ascending=False))
display(score_df[score_df['model']=='LogisticRegression'].groupby('model')['precision'].mean().to_frame().sort_values('precision', ascending=False))
display(score_df[score_df['model']=='LogisticRegression'].groupby('iter')['precision'].mean().to_frame().sort_values('precision', ascending=False))


In [ ]:
Parallel?
